In [4]:
#FLAPPY BIRD :D

In [2]:
#import libraries
import pygame
from pygame.locals import *
import os as os
import time as time
import random as random

os.chdir('/Users/benswart/Desktop/FlappyBird')

pygame 2.0.3 (SDL 2.0.16, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
#GLOBAL VARIABLES
screenWidth, screenHeight = 640, 480
pipeGap = 50
pipeWidth = 150
playerSize = 60
centerPointSize = [160, 320]
pipeOffSet = 10

class GameUtils():
    
    score = 0
    obstacles = []
    obstaclesPassed = []
    
    #generate pipes
    def generate_pipes(self, image):

        #generate pipe heights
        centerPoint = random.randint(centerPointSize[0], centerPointSize[1])
        bottomPipeHeight = screenHeight - centerPoint - pipeGap
        topPipeHeight = centerPoint - pipeGap

        #gen top image
        topPipe = pygame.transform.scale(image, (pipeWidth, topPipeHeight))
        topPipe = pygame.transform.rotate(topPipe, 180)
        
        #gen bottom image
        bottomPipe = pygame.transform.scale(image, (pipeWidth, bottomPipeHeight))
        
        #add top/bottom pipes to obstacles
        #                   top sprite  bot sprite  top starting x,y  bot starting x, y
        self.obstacles.append([topPipe, bottomPipe, [screenWidth, pipeOffSet*-1], [screenWidth, screenHeight - bottomPipeHeight + pipeOffSet]])
        self.obstaclesPassed.append(False)
        
    #move pipes
    def move_pipes(self):
        for index in range(len(self.obstacles) - 1):
            obstacle = self.obstacles[index]
            # change the x values for pipe to move to the left
            obstacle[2][0] -= 3
            obstacle[3][0] -= 3

            # remove obstacles from list once they have vanished
            if(obstacle[2][0] < -250):
                self.obstacles.pop(index)
                self.obstaclesPassed.pop(index)

    #check collisions
    def check_collisions(self, player):
        for obstacle in self.obstacles:
            
            #checking top pipe
            if(player.x_pos > obstacle[2][0] and player.x_pos < obstacle[2][0] + playerSize + pipeOffSet):
                if player.y_pos < obstacle[0].get_height() - (playerSize/2) - pipeOffSet:
                    pygame.quit()
                    return True
                
            #checking bottom pipe
            if(player.x_pos > obstacle[3][0] and player.x_pos < obstacle[3][0] + playerSize + pipeOffSet):
                #checking y
                if screenHeight - obstacle[1].get_height() - (playerSize/2) + pipeOffSet < player.y_pos:
                    pygame.quit()
                    return True
        
        return False
    
    def update_score(self, player):
        for index in range(len(self.obstacles)-1):
            obstacle = self.obstacles[index]
            if not self.obstaclesPassed[index] and player.x_pos > obstacle[2][0] + playerSize:
                self.score += 1
                self.obstaclesPassed[index] = True;
            
class Player():
    
    x_velcoity = 0
    y_velocity = 0
    isFlap = False
    jumpCount = 10
    score = 0
    
    def __init__(self, image, x_pos, y_pos):
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.image = image
        

In [4]:
def play_game():
    pygame.init()

    #Set font
    myfont = pygame.font.SysFont("monospace", 16)

    #Load / re-size sprites
    playerImg = pygame.image.load("bird.png")
    playerImg = pygame.transform.scale(playerImg, (playerSize, playerSize))
    backgroundImg = pygame.image.load("background.png")
    backgroundImg = pygame.transform.scale(backgroundImg, (screenWidth, screenHeight))
    pipeImg = pygame.image.load("pipe.png")
    
    #Create Player
    player = Player(playerImg, 200, 20)
    
    #Create Screen
    screen = pygame.display.set_mode((screenWidth, screenHeight))
    gameUtils = GameUtils()
        
    start = time.time() - 3
    run = True
    while run:
        pygame.time.delay(40)
        
        #Loop through the events        
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                run = False
        
        keys = pygame.key.get_pressed()
        
        if not player.isFlap:
            if player.y_pos <= 400:
                player.y_pos += 9.2
            
            if keys[pygame.K_SPACE]:
                player.isFlap = True
        else:
            if player.jumpCount >= 5:
                if(player.y_pos > 0):
                    player.y_pos -= ((player.jumpCount ** 2) / 5)
                player.jumpCount -= 1
            else:
                player.isFlap = False
                player.jumpCount = 10
        
        #Check if player has reached floor - end game
        if player.y_pos >= 400:
            pygame.quit()
            return
        
        #Generate obstacles
        now = time.time()
        if now - start > 3:
            gameUtils.generate_pipes(pipeImg)
            start = now
        
        #Move pipes 
        gameUtils.move_pipes()
                
        #Check collisons
        if gameUtils.check_collisions(player):
            return
        
        #Update score
        gameUtils.update_score(player)
        
        
        #------------------UPDATING SCREEN------------------#
        
        #Load background screen
        screen.fill(0)
        screen.blit(backgroundImg, (0, 0))
        
        #Load obstacles
        for obstacle in gameUtils.obstacles:
            screen.blit(obstacle[0], (obstacle[2][0], obstacle[2][1])) #load top
            screen.blit(obstacle[1], (obstacle[3][0], obstacle[3][1])) #load bottom
                        
        #Load player sprite
        screen.blit(player.image, (player.x_pos, player.y_pos))
        
        #Load score
        scoretext = myfont.render("Score: {0}".format(gameUtils.score), 1, (0,0,0))
        screen.blit(scoretext, (5, 10))
        
        #Update screen
        pygame.display.update()

#Start game
play_game()